In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into 80% training and 20% validation
)

train_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),  # Adjust to the input size expected by ResNet-150
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 381 images belonging to 4 classes.
Found 94 images belonging to 4 classes.


In [3]:
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Four classes

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
history = model.fit(train_generator, epochs=4, validation_data=validation_generator)


Epoch 1/4
12/12 [==============================] - 545s 43s/step - loss: 1.3469 - accuracy: 0.4987 - val_loss: 2963550208.0000 - val_accuracy: 0.1489
Epoch 2/4
12/12 [==============================] - 500s 42s/step - loss: 1.2145 - accuracy: 0.5039 - val_loss: 4417345536.0000 - val_accuracy: 0.1489
Epoch 3/4
12/12 [==============================] - 491s 42s/step - loss: 1.1010 - accuracy: 0.5118 - val_loss: 166771456.0000 - val_accuracy: 0.3404
Epoch 4/4
12/12 [==============================] - 519s 43s/step - loss: 1.0370 - accuracy: 0.5459 - val_loss: 5966577.0000 - val_accuracy: 0.1489


In [8]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)


Found 475 images belonging to 4 classes.
15/15 [==============================] - 99s 7s/step - loss: 5861107.0000 - accuracy: 0.1495


In [15]:
import tensorflow_model_optimization as tfmot

# Annotate the model for quantization-aware training
quant_annotate_model = tfmot.quantization.keras.quantize_annotate_model(model)


In [16]:
# Compile the quantization-aware model
quant_annotate_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
# Train the quantization-aware model
history = quant_annotate_model.fit(train_generator, epochs=4, validation_data=validation_generator)


Epoch 1/4
12/12 [==============================] - 531s 41s/step - loss: 1.3619 - accuracy: 0.4856 - val_loss: 11541002910229856256.0000 - val_accuracy: 0.1489
Epoch 2/4
12/12 [==============================] - 514s 43s/step - loss: 1.1921 - accuracy: 0.5118 - val_loss: 191427946513170432.0000 - val_accuracy: 0.2128
Epoch 3/4
12/12 [==============================] - 515s 43s/step - loss: 1.1454 - accuracy: 0.5249 - val_loss: 618721705984.0000 - val_accuracy: 0.2128
Epoch 4/4
12/12 [==============================] - 524s 44s/step - loss: 1.0599 - accuracy: 0.5827 - val_loss: 145660016.0000 - val_accuracy: 0.1489
